In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt


In [3]:
data_train = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_train_tech_nonscaled.parquet')

data_test = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_test_tech_nonscaled.parquet')

firm_data = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/Firm_variables/daily_firm_data_median_new.parquet')

# Top features
top_features_c = ['cp_flag',	'Ticker',	'date',	'moneyness',	'impl_volatility', 'prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock', '1Y_bond']
top_features_p = ['cp_flag',	'Ticker',	'date',	'moneyness',	'impl_volatility', 'prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock']

# # List of columns to drop
columns_to_drop = ['trading_days_till_exp'] + list(firm_data.columns[2:]) + ['moneyness_squared', 'tau_squared', 'moneyness_tau', 'best_offer_option', 'best_bid_option']
# # columns_to_drop = ['trading_days_till_exp']


# # Drop columns from datasets if they exist
data_train = data_train.drop(columns=columns_to_drop, errors='ignore')
data_test = data_test.drop(columns=columns_to_drop, errors='ignore')

# Ensure that data_val and data_test have the same column order as data_train
data_test = data_test[data_train.columns]

data_trains = data_train.copy()

# data_train = data_trains[data_trains['date'] < '2020-01-01']
# data_validate = data_trains[data_trains['date'] >= '2020-01-01']

In [4]:
data_train

,cp_flag,Ticker,date,moneyness,impl_volatility,T,prev_day_iv,prev2_day_iv,volume_option,spread_option,...,gold_price,reces_indi,10Y_RIR,1Y_bond,2Y_bond,OPEN_vix,HIGH_vix,LOW_vix,CLOSE_vix,spread_vix
0,P,AAPL,2019-01-02,-1.860,0.558181,0.011905,0.505761,0.000000,786,0.01,...,449.000000,0.130000,2.66,2.5342,2.5442,31.72,31.74,28.57,28.57,3.17
1,P,AAPL,2019-01-02,-1.682,0.541292,0.011905,0.495051,0.000000,982,0.01,...,449.000000,0.130000,2.66,2.5342,2.5442,31.72,31.74,28.57,28.57,3.17
2,P,AAPL,2019-01-02,-1.505,0.525111,0.011905,0.484170,0.000000,2614,0.01,...,449.000000,0.130000,2.66,2.5342,2.5442,31.72,31.74,28.57,28.57,3.17
3,P,AAPL,2019-01-02,-1.330,0.507597,0.011905,0.473299,0.000000,7308,0.01,...,449.000000,0.130000,2.66,2.5342,2.5442,31.72,31.74,28.57,28.57,3.17
4,P,AAPL,2019-01-02,-0.895,0.468786,0.011905,0.455607,0.000000,11884,0.02,...,449.000000,0.130000,2.66,2.5342,2.5442,31.72,31.74,28.57,28.57,3.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145306,C,TSLA,2020-12-31,1.467,0.623205,0.023810,0.554273,0.486818,1054,0.10,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26
145307,C,TSLA,2020-12-31,1.535,0.629719,0.023810,0.554273,0.486818,25829,0.06,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26
145308,C,TSLA,2020-12-31,1.670,0.639880,0.023810,0.584788,0.511182,1927,0.04,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26
145309,C,TSLA,2020-12-31,1.804,0.648805,0.023810,0.601543,0.523462,3447,0.06,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26


In [5]:
# import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, r2_score

# # Assuming 'data_train' is already defined and loaded
# data_train_c = data_train[data_train['cp_flag'] == 'C']
# data_train_p = data_train[data_train['cp_flag'] == 'P']

# def prepare_data(data, option_type):
#     """
#     Prepare the data and train Random Forest for a given dataset.
#     Prints the in-sample RMSE and R².
    
#     Parameters:
#     data (pd.DataFrame): The input dataset.
#     option_type (str): Call or Put option type for labeling the print output.
#     """
#     # Prepare the train data
#     X = data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Features
#     y = data['impl_volatility']  # Target variable
    
#     # Initialize the RandomForestRegressor
#     rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

#     # Train the model
#     rf_model.fit(X, y)

#     # In-sample (training set) predictions
#     y_train_pred = rf_model.predict(X)

#     # Evaluate In-Sample Performance (on Training Data)
#     rmse_train = np.sqrt(mean_squared_error(y, y_train_pred))
#     r2_train = r2_score(y, y_train_pred)

#     # Print the results
#     print(f"\nIn-Sample Performance for {option_type} Options:")
#     print(f"Random Forest RMSE (Training): {rmse_train:.4f}")
#     print(f"Random Forest R² (Training): {r2_train:.4f}")


# # Call the function for Call options data
# prepare_data(data_train_c, 'Call')

# # Call the function for Put options data
# prepare_data(data_train_p, 'Put')


# data_test_c = data_test[data_test['cp_flag'] == 'C']
# data_test_p = data_test[data_test['cp_flag'] == 'P']


# y_test_c = data_test_c['impl_volatility']
# y_test_p = data_test_p['impl_volatility']

# X_test_c = data_test_c.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])
# X_test_p = data_test_p.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])

# # Predict on the test set
# y_pred = rf_model.predict(X_test_c)

# # Evaluate model performance
# rmse_rf = np.sqrt(mean_squared_error(y_test_c, y_pred))
# r2_rf = r2_score(y_test_c, y_pred)

# print(f"Random Forest RMSE: {rmse_rf:.4f}")
# print(f"Random Forest R²: {r2_rf:.4f}")


In [6]:
# import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, r2_score

# # Assuming 'data_train' and 'data_test' are already defined and loaded

# # Prepare train data for Call and Put options
# data_train_c = data_train[data_train['cp_flag'] == 'C']
# data_train_p = data_train[data_train['cp_flag'] == 'P']

# # Prepare test data for Call and Put options
# data_test_c = data_test[data_test['cp_flag'] == 'C']
# data_test_p = data_test[data_test['cp_flag'] == 'P']

# def prepare_data(train_data, test_data, option_type):
#     """
#     Prepare the data, train Random Forest, and evaluate both in-sample and out-of-sample performance.
    
#     Parameters:
#     train_data (pd.DataFrame): The training dataset.
#     test_data (pd.DataFrame): The testing dataset.
#     option_type (str): Call or Put option type for labeling the print output.
#     """
#     # Prepare the train and test data
#     X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
#     y_train = train_data['impl_volatility']  # Training target
    
#     X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
#     y_test = test_data['impl_volatility']  # Test target
    
#     # Initialize the RandomForestRegressor
#     rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

#     # Train the model
#     rf_model.fit(X_train, y_train)

#     # In-sample (training set) predictions
#     y_train_pred = rf_model.predict(X_train)

#     # Out-of-sample (test set) predictions
#     y_test_pred = rf_model.predict(X_test)

#     # Evaluate In-Sample Performance (on Training Data)
#     rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
#     r2_train = r2_score(y_train, y_train_pred)

#     # Evaluate Out-of-Sample Performance (on Test Data)
#     rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
#     r2_test = r2_score(y_test, y_test_pred)

#     # Print the results
#     print(f"\nPerformance for {option_type} Options:")
#     print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
#     print(f"In-Sample R² (Training): {r2_train:.4f}")
#     print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
#     print(f"Out-of-Sample R² (Test): {r2_test:.4f}")

# # Call the function for Call options data
# prepare_data(data_train_c, data_test_c, 'Call')

# # Call the function for Put options data
# prepare_data(data_train_p, data_test_p, 'Put')


In [8]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# # Top features for Call and Put options
# top_features_c = ['cp_flag', 'Ticker', 'date', 'moneyness', 'impl_volatility', 'prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock', '1Y_bond']
# top_features_p = ['cp_flag', 'Ticker', 'date', 'moneyness', 'impl_volatility', 'prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock']

top_features_c = ['cp_flag', 'Ticker', 'date', 'impl_volatility', 'prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock', '1Y_bond', 'spread_stock', 'spread_vix', 'moneyness','5_day_rolling_return_stock', 'RET','PRC_actual','CLOSE_vix','OPEN_vix','reces_indi' ]
# top_features_c =['prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock', '1Y_bond', 'spread_stock', 'spread_vix', 'moneyness', '5_day_rolling_return_stock']
# top_features_p =['prev_day_iv', 'T', 'vol_stock', 'prev2_day_iv', 'hi-lo_stock', '5_day_rolling_return_stock', 'RET', 'moneyness', 'PRC_actual', 'CLOSE_vix']

# # Prepare train data for Call and Put options
data_train_c = data_train[data_train['cp_flag'] == 'C'][top_features_c]
data_train_p = data_train[data_train['cp_flag'] == 'P'][top_features_p]

# # # Prepare validation data for Call and Put options
# # data_validate_c = data_validate[data_validate['cp_flag'] == 'C'][top_features_c]
# # data_validate_p = data_validate[data_validate['cp_flag'] == 'P'][top_features_p]

# # Prepare test data for Call and Put options
data_test_c = data_test[data_test['cp_flag'] == 'C'][top_features_c]
data_test_p = data_test[data_test['cp_flag'] == 'P'][top_features_p]

In [9]:
data_test_c

,prev_day_iv,T,vol_stock,prev2_day_iv,hi-lo_stock,1Y_bond,spread_stock,spread_vix,moneyness,5_day_rolling_return_stock
0,0.337114,0.019841,95485520.0,0.325650,2.59002,0.1074,0.00999,1.92,-0.475,0.013952
1,0.332640,0.019841,95485520.0,0.312515,2.59002,0.1074,0.00999,1.92,-0.345,0.013952
2,0.330864,0.019841,95485520.0,0.308907,2.59002,0.1074,0.00999,1.92,-0.216,0.013952
3,0.329431,0.019841,95485520.0,0.306933,2.59002,0.1074,0.00999,1.92,-0.088,0.013952
4,0.328318,0.019841,95485520.0,0.305940,2.59002,0.1074,0.00999,1.92,0.039,0.013952
...,...,...,...,...,...,...,...,...,...,...
36347,0.718058,0.007937,20025526.0,0.773147,40.57984,0.4878,0.51990,2.04,1.556,0.209492
36348,0.735985,0.007937,20025526.0,0.789361,40.57984,0.4878,0.51990,2.04,1.658,0.209492
36349,0.754168,0.007937,20025526.0,0.807648,40.57984,0.4878,0.51990,2.04,1.760,0.209492
36350,0.773210,0.007937,20025526.0,0.821686,40.57984,0.4878,0.51990,2.04,1.861,0.209492


# All features

In [47]:
# Prepare train data for Call and Put options
data_train_c = data_train[data_train['cp_flag'] == 'C']
data_train_p = data_train[data_train['cp_flag'] == 'P']

# # Prepare validation data for Call and Put options
# data_validate_c = data_validate[data_validate['cp_flag'] == 'C'][top_features_c]
# data_validate_p = data_validate[data_validate['cp_flag'] == 'P'][top_features_p]

# Prepare test data for Call and Put options
data_test_c = data_test[data_test['cp_flag'] == 'C']
data_test_p = data_test[data_test['cp_flag'] == 'P']

In [33]:
# import time
# import itertools
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, r2_score
# import numpy as np

# def prepare_data_with_gridsearch(train_data, validate_data, test_data, option_type, verbose=True):

#     """
#     Prepare the data, train Random Forest by manually tuning hyperparameters with validation data, 
#     and evaluate both in-sample and out-of-sample performance.
    
#     Parameters:
#     train_data (pd.DataFrame): The training dataset (year 1).
#     validate_data (pd.DataFrame): The validation dataset (year 2).
#     test_data (pd.DataFrame): The testing dataset.
#     option_type (str): Call or Put option type for labeling the print output.
#     verbose (bool): If True, prints progress information for hyperparameter tuning.
#     """
#     # Prepare the train, validation, and test data
#     X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
#     y_train = train_data['impl_volatility']  # Training target
    
#     X_validate = validate_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Validation features
#     y_validate = validate_data['impl_volatility']  # Validation target
    
#     X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
#     y_test = test_data['impl_volatility']  # Test target
    
#     # Define the hyperparameter grid
#     param_grid = {
#         'max_features': [5, 10, 15],                 # Use fewer features to reduce model complexity
#         'n_estimators': [100, 200, 300],             # Increase number of trees for better generalization
#         'min_samples_leaf': [5, 10, 15],             # Ensure minimum leaf size is larger to reduce overfitting
#         'min_samples_split': [2, 5],                 # Ensure reasonable splits, prevent unnecessary splits
#         'max_depth': [5, 10, 15, 20]                 # Limit the depth of trees to prevent overfitting
#     }
    
#     # Generate all combinations of hyperparameters
#     param_combinations = list(itertools.product(
#         param_grid['max_features'], 
#         param_grid['n_estimators'], 
#         param_grid['min_samples_leaf'], 
#         param_grid['min_samples_split'], 
#         param_grid['max_depth']
#     ))
    
#     total_combinations = len(param_combinations)
    
#     # Initialize variables to store the best model and best score
#     best_rmse_val = np.inf
#     best_params = None
#     best_rf_model = None
    
#     print(f"Running manual hyperparameter tuning for {option_type} Options...")
    
#     # Iterate over all hyperparameter combinations with progress tracking
#     for i, (max_features, n_estimators, min_samples_leaf, min_samples_split, max_depth) in enumerate(param_combinations):
#         start_time = time.time()
        
#         # Initialize the RandomForestRegressor with the current set of hyperparameters
#         rf_model = RandomForestRegressor(
#             max_features=max_features, 
#             n_estimators=n_estimators,
#             min_samples_leaf=min_samples_leaf,
#             min_samples_split=min_samples_split,
#             max_depth=max_depth
#         )
    
#         # Train the model on the training data (Year 1)
#         rf_model.fit(X_train, y_train)
    
#         # Validate the model on the validation data (Year 2)
#         y_val_pred = rf_model.predict(X_validate)
#         rmse_val = np.sqrt(mean_squared_error(y_validate, y_val_pred))
    
#         # Track the best performing hyperparameters based on validation RMSE
#         if rmse_val < best_rmse_val:
#             best_rmse_val = rmse_val
#             best_params = {
#                 'max_features': max_features,
#                 'n_estimators': n_estimators,
#                 'min_samples_leaf': min_samples_leaf,
#                 'min_samples_split': min_samples_split,
#                 'max_depth': max_depth
#             }
#             best_rf_model = rf_model
    
#         # Verbose output to track progress
#         if verbose:
#             elapsed_time = time.time() - start_time
#             print(f"Combination {i + 1}/{total_combinations} completed in {elapsed_time:.2f} seconds.")
#             print(f"Current RMSE (Validation): {rmse_val:.4f}")
#             print(f"Best RMSE so far: {best_rmse_val:.4f}")
    
#     print(f"\nBest Parameters for {option_type} Options: {best_params}")
    
#     # After finding the best hyperparameters, evaluate performance on the test data
#     # In-sample (training set) predictions
#     y_train_pred = best_rf_model.predict(X_train)
    
#     # Validate the predictions
#     y_val_pred = best_rf_model.predict(X_validate)
    
#     # Out-of-sample (test set) predictions
#     y_test_pred = best_rf_model.predict(X_test)
    
#     # Evaluate In-Sample Performance (on Training Data)
#     rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
#     r2_train = r2_score(y_train, y_train_pred)
    
#     # Evaluate Validation Set Performance
#     rmse_val = np.sqrt(mean_squared_error(y_validate, y_val_pred))
#     r2_val = r2_score(y_validate, y_val_pred)
    
#     # Evaluate Out-of-Sample Performance (on Test Data)
#     rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
#     r2_test = r2_score(y_test, y_test_pred)
    
#     # Print the results
#     print(f"\nPerformance for {option_type} Options:")
#     print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
#     print(f"In-Sample R² (Training): {r2_train:.4f}")
#     print(f"Validation RMSE: {rmse_val:.4f}")
#     print(f"Validation R²: {r2_val:.4f}")
#     print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
#     print(f"Out-of-Sample R² (Test): {r2_test:.4f}")

# prepare_data_with_gridsearch(data_train_c, data_validate_c, data_test_c, 'Call')

# prepare_data_with_gridsearch(data_train_p, data_validate_p, data_test_p, 'Put')

In [34]:
# import time
# import itertools
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, r2_score
# import numpy as np

# def prepare_data_with_gridsearch(train_data, validate_data, test_data, option_type, verbose=True):
#     """
#     Prepare the data, perform hyperparameter tuning using Year 1 (train) and Year 2 (validation),
#     retrain the model on Year 1 + Year 2, and evaluate on Year 3 (test).
    
#     Parameters:
#     train_data (pd.DataFrame): The training dataset (Year 1).
#     validate_data (pd.DataFrame): The validation dataset (Year 2).
#     test_data (pd.DataFrame): The testing dataset (Year 3).
#     option_type (str): Call or Put option type for labeling the print output.
#     verbose (bool): If True, prints progress information for hyperparameter tuning.
#     """
#     # Prepare the train, validation, and test data
#     X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features (Year 1)
#     y_train = train_data['impl_volatility']  # Training target (Year 1)

#     X_validate = validate_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Validation features (Year 2)
#     y_validate = validate_data['impl_volatility']  # Validation target (Year 2)
    
#     X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features (Year 3)
#     y_test = test_data['impl_volatility']  # Test target (Year 3)

#     # Define the hyperparameter grid
#     param_grid = {
#         'n_estimators': [50, 75, 100],             # Increase number of trees for better generalization
#         'min_samples_leaf': [1,2, 4, 7],             # Ensure minimum leaf size is larger to reduce overfitting
#         'min_samples_split': [2, 7, 12],                 # Ensure reasonable splits, prevent unnecessary splits
#         'max_depth': [10, 20, None]                 # Limit the depth of trees to prevent overfitting
#     }
#     # Best params Call
#     # param_grid = {'max_features': [20], 'n_estimators': [50], 'min_samples_leaf': [7], 'min_samples_split': [2], 'max_depth': [None]}

#     # Best params Put


#     # Generate all combinations of hyperparameters
#     param_combinations = list(itertools.product(
#         param_grid['n_estimators'], 
#         param_grid['min_samples_leaf'], 
#         param_grid['min_samples_split'], 
#         param_grid['max_depth']
#     ))

#     total_combinations = len(param_combinations)
    
#     # Initialize variables to store the best model and best score
#     best_rmse_val = np.inf
#     best_params = None
#     best_rf_model = None

#     print(f"Running manual hyperparameter tuning for {option_type} Options...")
    
#     # Iterate over all hyperparameter combinations with progress tracking
#     for i, (n_estimators, min_samples_leaf, min_samples_split, max_depth) in enumerate(param_combinations):
#         start_time = time.time()
        
#         # Initialize the RandomForestRegressor with the current set of hyperparameters
#         rf_model = RandomForestRegressor(
#             n_estimators=n_estimators,
#             min_samples_leaf=min_samples_leaf,
#             min_samples_split=min_samples_split,
#             max_depth=max_depth
#         )

#         # Train the model on the training data (Year 1)
#         rf_model.fit(X_train, y_train)

#         # Validate the model on the validation data (Year 2)
#         y_val_pred = rf_model.predict(X_validate)
#         rmse_val = np.sqrt(mean_squared_error(y_validate, y_val_pred))

#         # Track the best performing hyperparameters based on validation RMSE
#         if rmse_val < best_rmse_val:
#             best_rmse_val = rmse_val
#             best_params = {
#                 'n_estimators': n_estimators,
#                 'min_samples_leaf': min_samples_leaf,
#                 'min_samples_split': min_samples_split,
#                 'max_depth': max_depth
#             }
#             best_rf_model = rf_model

#         # Verbose output to track progress
#         if verbose:
#             elapsed_time = time.time() - start_time
#             print(f"Combination {i + 1}/{total_combinations} completed in {elapsed_time:.2f} seconds.")
#             print(f"Current RMSE (Validation): {rmse_val:.4f}")
#             print(f"Best RMSE so far: {best_rmse_val:.4f}")
    
#     print(f"\nBest Parameters for {option_type} Options: {best_params}")
    
#     # Retrain the model on combined Year 1 (train) and Year 2 (validation)
#     print("Retraining the model on Year 1 and Year 2 combined...")
#     X_combined = pd.concat([X_train, X_validate])
#     y_combined = pd.concat([y_train, y_validate])
#     best_rf_model.fit(X_combined, y_combined)

#     # In-sample (combined Year 1 + Year 2) predictions
#     y_combined_pred = best_rf_model.predict(X_combined)

#     # Evaluate In-Sample Performance (on combined Year 1 + Year 2)
#     rmse_combined = np.sqrt(mean_squared_error(y_combined, y_combined_pred))
#     r2_combined = r2_score(y_combined, y_combined_pred)
    
#     print(f"\nIn-Sample Performance for {option_type} Options (Year 1 + Year 2):")
#     print(f"RMSE (Training + Validation): {rmse_combined:.4f}")
#     print(f"R² (Training + Validation): {r2_combined:.4f}")

#     # After retraining, evaluate performance on the test data (Year 3)
#     y_test_pred = best_rf_model.predict(X_test)

#     # Evaluate Out-of-Sample Performance (on Test Data)
#     rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
#     r2_test = r2_score(y_test, y_test_pred)

#     # Print the final results
#     print(f"\nPerformance on Test Data (Year 3) for {option_type} Options:")
#     print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
#     print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# # Call the function for Call options data
# prepare_data_with_gridsearch(data_train_c, data_validate_c, data_test_c, 'Call')

# # Call the function for Put options data
# prepare_data_with_gridsearch(data_train_p, data_validate_p, data_test_p, 'Put')


In [35]:
# import time
# import itertools
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, r2_score
# import numpy as np

# def prepare_data_with_gridsearch(train_data, validate_data, test_data, option_type, verbose=True):
#     """
#     Prepare the data, perform hyperparameter tuning using Year 1 (train) and Year 2 (validation),
#     retrain the model on Year 1 + Year 2, and evaluate on Year 3 (test).
    
#     Parameters:
#     train_data (pd.DataFrame): The training dataset (Year 1).
#     validate_data (pd.DataFrame): The validation dataset (Year 2).
#     test_data (pd.DataFrame): The testing dataset (Year 3).
#     option_type (str): Call or Put option type for labeling the print output.
#     verbose (bool): If True, prints progress information for hyperparameter tuning.
#     """
#     # Prepare the train, validation, and test data
#     X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features (Year 1)
#     y_train = train_data['impl_volatility']  # Training target (Year 1)

#     X_validate = validate_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Validation features (Year 2)
#     y_validate = validate_data['impl_volatility']  # Validation target (Year 2)
    
#     X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features (Year 3)
#     y_test = test_data['impl_volatility']  # Test target (Year 3)

#     # Define the hyperparameter grid
#     param_grid = {
#     'n_estimators': [100, 200, 300],
#     'min_samples_leaf': [10, 20, 30],
#     'min_samples_split': [5, 10, 15],
#     'max_depth': [5, 10, 15]  # Limit depth to avoid overfitting            # Limit the depth of trees to prevent overfitting
#     }
#     # Best params Call
#     # param_grid = {'max_features': [20], 'n_estimators': [50], 'min_samples_leaf': [7], 'min_samples_split': [2], 'max_depth': [None]}

#     # Best params Put


#     # Generate all combinations of hyperparameters
#     param_combinations = list(itertools.product(
#         param_grid['n_estimators'], 
#         param_grid['min_samples_leaf'], 
#         param_grid['min_samples_split'], 
#         param_grid['max_depth']
#     ))

#     total_combinations = len(param_combinations)
    
#     # Initialize variables to store the best model and best score
#     best_rmse_val = np.inf
#     best_params = None
#     best_rf_model = None

#     print(f"Running manual hyperparameter tuning for {option_type} Options...")
    
#     # Iterate over all hyperparameter combinations with progress tracking
#     for i, (n_estimators, min_samples_leaf, min_samples_split, max_depth) in enumerate(param_combinations):
#         start_time = time.time()
        
#         # Initialize the RandomForestRegressor with the current set of hyperparameters
#         rf_model = RandomForestRegressor(
#             n_estimators=n_estimators,
#             min_samples_leaf=min_samples_leaf,
#             min_samples_split=min_samples_split,
#             max_depth=max_depth
#         )

#         # Train the model on the training data (Year 1)
#         rf_model.fit(X_train, y_train)

#         # Validate the model on the validation data (Year 2)
#         y_val_pred = rf_model.predict(X_validate)
#         rmse_val = np.sqrt(mean_squared_error(y_validate, y_val_pred))

#         # Track the best performing hyperparameters based on validation RMSE
#         if rmse_val < best_rmse_val:
#             best_rmse_val = rmse_val
#             best_params = {
#                 'n_estimators': n_estimators,
#                 'min_samples_leaf': min_samples_leaf,
#                 'min_samples_split': min_samples_split,
#                 'max_depth': max_depth
#             }
#             best_rf_model = rf_model

#         # Verbose output to track progress
#         if verbose:
#             elapsed_time = time.time() - start_time
#             print(f"Combination {i + 1}/{total_combinations} completed in {elapsed_time:.2f} seconds.")
#             print(f"Current RMSE (Validation): {rmse_val:.4f}")
#             print(f"Best RMSE so far: {best_rmse_val:.4f}")
    
#     print(f"\nBest Parameters for {option_type} Options: {best_params}")
    
#     # Retrain the model on combined Year 1 (train) and Year 2 (validation)
#     print("Retraining the model on Year 1 and Year 2 combined...")
#     X_combined = pd.concat([X_train, X_validate])
#     y_combined = pd.concat([y_train, y_validate])
#     best_rf_model.fit(X_combined, y_combined)

#     # In-sample (combined Year 1 + Year 2) predictions
#     y_combined_pred = best_rf_model.predict(X_combined)

#     # Evaluate In-Sample Performance (on combined Year 1 + Year 2)
#     rmse_combined = np.sqrt(mean_squared_error(y_combined, y_combined_pred))
#     r2_combined = r2_score(y_combined, y_combined_pred)
    
#     print(f"\nIn-Sample Performance for {option_type} Options (Year 1 + Year 2):")
#     print(f"RMSE (Training + Validation): {rmse_combined:.4f}")
#     print(f"R² (Training + Validation): {r2_combined:.4f}")

#     # After retraining, evaluate performance on the test data (Year 3)
#     y_test_pred = best_rf_model.predict(X_test)

#     # Evaluate Out-of-Sample Performance (on Test Data)
#     rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
#     r2_test = r2_score(y_test, y_test_pred)

#     # Print the final results
#     print(f"\nPerformance on Test Data (Year 3) for {option_type} Options:")
#     print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
#     print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# # Call the function for Call options data
# prepare_data_with_gridsearch(data_train_c, data_validate_c, data_test_c, 'Call')

# # Call the function for Put options data
# prepare_data_with_gridsearch(data_train_p, data_validate_p, data_test_p, 'Put')


In [36]:
# def prepare_data_with_gridsearch(train_data, validate_data, test_data, option_type):
#     """
#     Prepare the data, train Random Forest using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
    
#     Parameters:
#     train_data (pd.DataFrame): The training dataset.
#     test_data (pd.DataFrame): The testing dataset.
#     option_type (str): Call or Put option type for labeling the print output.
#     """
#     # Prepare the train and test data
#     X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
#     y_train = train_data['impl_volatility']  # Training target

#     # Prepare the validation data
#     X_validate = validate_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
#     y_validate = validate_data['impl_volatility']  # Training target
    
#     X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
#     y_test = test_data['impl_volatility']  # Test target
    
#     # # Define the parameter grid for GridSearchCV
#     # param_grid = {
#     #     'n_estimators': [100, 200],      # Number of trees in the forest
#     #     'max_depth': [10, 20, None],     # Maximum depth of the tree
#     #     'min_samples_split': [2, 5],     # Minimum number of samples required to split an internal node
#     #     'min_samples_leaf': [1, 2],      # Minimum number of samples required to be at a leaf node
#     #     'bootstrap': [True, False]       # Whether bootstrap samples are used when building trees
#     # }

#     param_grid = {
#     'max_features': [5, 10, 15],                 # Use fewer features to reduce model complexity
#     'n_estimators': [100, 200, 300],             # Increase number of trees for better generalization
#     'min_samples_leaf': [5, 10, 15],             # Ensure minimum leaf size is larger to reduce overfitting
#     'min_samples_split': [2, 5],                 # Ensure reasonable splits, prevent unnecessary splits
#     'max_depth': [5, 10, 15, 20]                 # Limit the depth of trees to prevent overfitting
#     }   

#     # Initialize the RandomForestRegressor
#     rf_model = RandomForestRegressor()

#     # Initialize GridSearchCV
#     grid_search = GridSearchCV(estimator=rf_model, param_grid=(param_grid), 
#                                cv=2, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

#     # Train the model using GridSearchCV
#     print(f"Running GridSearchCV for {option_type} Options...")
#     grid_search.fit(X_train, y_train)

#     # Get the best parameters and the best estimator
#     best_rf_model = grid_search.best_estimator_
#     print(f"\nBest Parameters for {option_type} Options: {grid_search.best_params_}")

#     # In-sample (training set) predictions
#     y_train_pred = best_rf_model.predict(X_train)

#     # Validate the predictions
#     y_val_pred = best_rf_model.predict(X_validate)

#     # Out-of-sample (test set) predictions
#     y_test_pred = best_rf_model.predict(X_test)

#     # Evaluate In-Sample Performance (on Training Data)
#     rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
#     r2_train = r2_score(y_train, y_train_pred)

#     # Evaluate Validation Set Performance
#     rmse_val = np.sqrt(mean_squared_error(y_validate, y_val_pred))
#     r2_val = r2_score(y_validate, y_val_pred)

#     # Evaluate Out-of-Sample Performance (on Test Data)
#     rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
#     r2_test = r2_score(y_test, y_test_pred)

#     # Print the results
#     print(f"\nPerformance for {option_type} Options:")
#     print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
#     print(f"In-Sample R² (Training): {r2_train:.4f}")
#     print(f"Validation RMSE: {rmse_val:.4f}")
#     print(f"Validation R²: {r2_val:.4f}")
#     print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
#     print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# # Call the function for Call options data
# prepare_data_with_gridsearch(data_train_c, data_validate_c, data_test_c, 'Call')

# # Call the function for Put options data
# prepare_data_with_gridsearch(data_train_p, data_validate_p, data_test_p, 'Put')


In [10]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

# Assuming 'data_train' and 'data_test' are already defined and loaded

# Prepare train data for Call and Put options
data_train_c = data_train[data_train['cp_flag'] == 'C']
data_train_p = data_train[data_train['cp_flag'] == 'P']

# Prepare test data for Call and Put options
data_test_c = data_test[data_test['cp_flag'] == 'C']
data_test_p = data_test[data_test['cp_flag'] == 'P']

def prepare_data_with_gridsearch(train_data, test_data, option_type):
    """
    Prepare the data, train Random Forest using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
    
    Parameters:
    train_data (pd.DataFrame): The training dataset.
    test_data (pd.DataFrame): The testing dataset.
    option_type (str): Call or Put option type for labeling the print output.
    
    Returns:
    feature_importances (pd.Series): The importance of features from the best model.
    """
    # Prepare the train and test data
    X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
    y_train = train_data['impl_volatility']  # Training target
    
    X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
    y_test = test_data['impl_volatility']  # Test target

    # Define the parameter grid for GridSearchCV for Call and Put separately
    if option_type == 'Call':
        param_grid = {'max_features': [15], 'n_estimators': [150], 'min_samples_leaf': [4], 'min_samples_split': [2], 'max_depth': [15]}  # Call
    else:
        param_grid = {'max_features': [15], 'n_estimators': [150], 'min_samples_leaf': [1], 'min_samples_split': [3], 'max_depth': [15]}  # Put

    # Initialize the RandomForestRegressor
    rf_model = RandomForestRegressor(random_state=42, max_samples=0.8)

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

    # Train the model using GridSearchCV
    print(f"Running GridSearchCV for {option_type} Options...")
    grid_search.fit(X_train, y_train)

    # Get the best parameters and the best estimator
    best_rf_model = grid_search.best_estimator_
    print(f"\nBest Parameters for {option_type} Options: {grid_search.best_params_}")

    # In-sample (training set) predictions
    y_train_pred = best_rf_model.predict(X_train)

    # Out-of-sample (test set) predictions
    y_test_pred = best_rf_model.predict(X_test)

    # Evaluate In-Sample Performance (on Training Data)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    r2_train = r2_score(y_train, y_train_pred)

    # Evaluate Out-of-Sample Performance (on Test Data)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)

    # Print the results
    print(f"\nPerformance for {option_type} Options:")
    print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
    print(f"In-Sample R² (Training): {r2_train:.4f}")
    print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
    print(f"Out-of-Sample R² (Test): {r2_test:.4f}")

    # Extract and return feature importances
    feature_importances = pd.Series(best_rf_model.feature_importances_, index=X_train.columns)
    return feature_importances

# Call the function for Call options data
feature_importance_c = prepare_data_with_gridsearch(data_train_c, data_test_c, 'Call')

# Call the function for Put options data
feature_importance_p = prepare_data_with_gridsearch(data_train_p, data_test_p, 'Put')


# Average feature importance plot
def plot_average_feature_importance(feature_importance_call, feature_importance_put, top_n=5):
    """
    Create a bar plot showing the top N average feature importance of both Call and Put options.

    Parameters:
    feature_importance_call (pd.Series): Feature importance for Call options.
    feature_importance_put (pd.Series): Feature importance for Put options.
    top_n (int): Number of top features to plot.
    """
    # Combine the importance of features from both Call and Put
    combined_importance = pd.concat([feature_importance_call, feature_importance_put], axis=1, keys=["Call", "Put"]).fillna(0)

    # Calculate the average importance
    combined_importance["Average"] = combined_importance.mean(axis=1)

    # Select the top N features based on average importance
    top_features = combined_importance["Average"].sort_values(ascending=False).head(top_n)

    # Plot the top N average feature importance
    plt.figure(figsize=(10, 6))
    ax = top_features.sort_values(ascending=True).plot(kind='barh', color='blue', alpha=0.55, edgecolor='black', linewidth=1)
    plt.grid(True)  # Add grid
    ax.set_title('RF (Put and Call)')
    # ax.set_xlabel('Importance')  # Add x-label
    plt.show()

# Example usage: Plot the top 5 features from the 15 chosen
# plot_average_feature_importance(feature_importance_c, feature_importance_p, top_n=5)


Running GridSearchCV for Call Options...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  33.0s
[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  34.9s
[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  35.6s
[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  35.7s
[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  36.8s

Best Parameters for Call Options: {'max_depth': 15, 'max_features': 15, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 150}

Performance for Call Options:
In-Sample RMSE (Training): 0.0410
In-Sample R² (Training): 0.9875
Out-of-Sample RMSE (Test): 0.1356
Out-of-Sample R² (Test): 0.6769
Runnin

KeyboardInterrupt: 

In [41]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
import random

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

# Set the seed before training
set_seed(42)
# Assuming 'data_train' and 'data_test' are already defined and loaded

# Prepare train data for Call and Put options
data_train_c = data_train[data_train['cp_flag'] == 'C']
data_train_p = data_train[data_train['cp_flag'] == 'P']

# Prepare test data for Call and Put options
data_test_c = data_test[data_test['cp_flag'] == 'C']
data_test_p = data_test[data_test['cp_flag'] == 'P']

def prepare_data_with_gridsearch(train_data, test_data, option_type):
    """
    Prepare the data, train Random Forest using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
    
    Parameters:
    train_data (pd.DataFrame): The training dataset.
    test_data (pd.DataFrame): The testing dataset.
    option_type (str): Call or Put option type for labeling the print output.
    """
    # Prepare the train and test data
    X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
    y_train = train_data['impl_volatility']  # Training target
    
    X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
    y_test = test_data['impl_volatility']  # Test target
    
    # # Define the parameter grid for GridSearchCV
    # param_grid = {
    #     'n_estimators': [100, 200],      # Number of trees in the forest
    #     'max_depth': [10, 20, None],     # Maximum depth of the tree
    #     'min_samples_split': [2, 5],     # Minimum number of samples required to split an internal node
    #     'min_samples_leaf': [1, 2],      # Minimum number of samples required to be at a leaf node
    #     'bootstrap': [True, False]       # Whether bootstrap samples are used when building trees
    # }

    # Hyperparameters Best
    # param_grid = {
    #     'max_features': [5,10,15, 20, 25, 30],
    #     'max_depth': [3,4,5, 6, 7 ,8, 10],
    #     'n_estimators': [50, 100,150, 200,300, 400, 500, 600, 700],
    # } np.linspace(0, 1, 10)

    # param_grid = {
    # # 'n_estimators': [50, 75, 100, 200, 300],
    # # 'min_samples_leaf': [2, 7, 12, 20],
    # # 'min_samples_split': [1, 2,4, 5, 10, 15],
    # # 'max_depth': [10, 15, 20, None]  # Limit depth to avoid overfitting

    # 'max_features': [15],
    # 'n_estimators': [50, 75, 100],
    # 'min_samples_leaf': [1,2, 4],
    # 'min_samples_split': [2, 7, 12],
    # 'max_depth': [10, 20, None]  # Limit depth to avoid overfitting
    # }  

    param_grid = { 'max_features': [15], 'n_estimators': [150], 'min_samples_leaf': [4], 'min_samples_split': [2], 'max_depth': [15]} #Put
    param_grid = { 'max_features': [15], 'n_estimators': [150], 'min_samples_leaf': [1], 'min_samples_split': [3], 'max_depth': [15]} #Call

    # Initialize the RandomForestRegressor
    rf_model = RandomForestRegressor(random_state=42, max_samples=0.8)

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=rf_model, param_grid=(param_grid), 
                               cv=5, n_jobs=-1, verbose=2)

    # Train the model using GridSearchCV
    print(f"Running GridSearchCV for {option_type} Options...")
    grid_search.fit(X_train, y_train)

    # Get the best parameters and the best estimator
    best_rf_model = grid_search.best_estimator_
    print(f"\nBest Parameters for {option_type} Options: {grid_search.best_params_}")

    # In-sample (training set) predictions
    y_train_pred = best_rf_model.predict(X_train)

    # Out-of-sample (test set) predictions
    y_test_pred = best_rf_model.predict(X_test)

    # Evaluate In-Sample Performance (on Training Data)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    r2_train = r2_score(y_train, y_train_pred)

    # Evaluate Out-of-Sample Performance (on Test Data)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)

    # Print the results
    print(f"\nPerformance for {option_type} Options:")
    print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
    print(f"In-Sample R² (Training): {r2_train:.4f}")
    print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
    print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# Call the function for Call options data
prepare_data_with_gridsearch(data_train_c, data_test_c, 'Call')

# Call the function for Put options data
prepare_data_with_gridsearch(data_train_p, data_test_p, 'Put')


Running GridSearchCV for Put Options...
Fitting 5 folds for each of 1 candidates, totalling 5 fits


Python(37883) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  37.6s
[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  39.8s
[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  39.9s
[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  40.9s
[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  42.2s

Best Parameters for Put Options: {'max_depth': 15, 'max_features': 15, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 150}

Performance for Put Options:
In-Sample RMSE (Training): 0.0452
In-Sample R² (Training): 0.9868
Out-of-Sample RMSE (Test): 0.1312
Out-of-Sample R² (Test): 0.7172


In [11]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import random
import tensorflow as tf

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

# Set the seed before training
set_seed(42)

# Function to calculate Newey-West standard error
def newey_west_standard_error(errors, lag=1):
    """
    Computes the Newey-West standard error for the series of prediction errors.
    
    Parameters:
    - errors: Array-like of prediction errors
    - lag: Maximum lag to use for the Newey-West estimator (default is 1)
    
    Returns:
    - Newey-West standard error of the prediction errors
    """
    X = np.ones(len(errors))  # Constant term for OLS
    ols_model = sm.OLS(errors, X).fit(cov_type='HAC', cov_kwds={'maxlags': lag})
    nw_std_error = ols_model.bse[0]  # Newey-West standard error
    return nw_std_error

# Function to prepare data, train Random Forest using GridSearchCV, and evaluate performance
def prepare_data_with_gridsearch(train_data, test_data, option_type):
    """
    Prepare the data, train Random Forest using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
    
    Parameters:
    train_data (pd.DataFrame): The training dataset.
    test_data (pd.DataFrame): The testing dataset.
    option_type (str): Call or Put option type for labeling the print output.
    
    Returns:
    y_test (pd.Series): True values of test data for implied volatility
    y_test_pred (np.array): Predicted values of implied volatility for test data
    nw_std_error (float): Newey-West standard error for test errors
    """
    # Prepare the train and test data
    X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
    y_train = train_data['impl_volatility']  # Training target
    
    X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
    y_test = test_data['impl_volatility']  # Test target

    # Hyperparameter grid
    param_grid = { 'max_features': [15], 'n_estimators': [150], 'min_samples_leaf': [4], 'min_samples_split': [2], 'max_depth': [15]} if option_type == 'Put' else \
                 { 'max_features': [15], 'n_estimators': [150], 'min_samples_leaf': [1], 'min_samples_split': [3], 'max_depth': [15]}  # Call

    # Initialize the RandomForestRegressor
    rf_model = RandomForestRegressor(random_state=42, max_samples=0.8)

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
                               cv=5, n_jobs=-1, verbose=2)

    # Train the model using GridSearchCV
    print(f"Running GridSearchCV for {option_type} Options...")
    grid_search.fit(X_train, y_train)

    # Get the best estimator
    best_rf_model = grid_search.best_estimator_

    # Out-of-sample (test set) predictions
    y_test_pred = best_rf_model.predict(X_test)

    # Calculate test errors and Newey-West standard error
    errors_test = y_test - y_test_pred
    nw_std_error = newey_west_standard_error(errors_test, lag=1)

    print(f"{option_type} Options Newey-West Standard Error: {nw_std_error:.4f}")

    return y_test, y_test_pred, nw_std_error

# Call the function for Call options data
y_test_c, y_test_pred_c, calls_nw_std_error = prepare_data_with_gridsearch(data_train_c, data_test_c, 'Call')

# Call the function for Put options data
y_test_p, y_test_pred_p, puts_nw_std_error = prepare_data_with_gridsearch(data_train_p, data_test_p, 'Put')

# Calculate errors for Calls and Puts
calls_errors = y_test_c - y_test_pred_c
puts_errors = y_test_p - y_test_pred_p

# Pad the shorter error list with zeros to ensure both lists have the same length
max_length = max(len(calls_errors), len(puts_errors))

puts_errors_padded = np.pad(puts_errors, (0, max_length - len(puts_errors)), 'constant', constant_values=0)
calls_errors_padded = np.pad(calls_errors, (0, max_length - len(calls_errors)), 'constant', constant_values=0)

# Save the results to a DataFrame and CSV
dm_test_data = pd.DataFrame({
    'Put Errors': puts_errors_padded,
    'Call Errors': calls_errors_padded,
    'Put Newey-West Std Error': [puts_nw_std_error] * max_length,  # Constant value for all rows
    'Call Newey-West Std Error': [calls_nw_std_error] * max_length  # Constant value for all rows
})

# Specify the file path for the CSV
file_path = '/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/performence_evaluation/dm_test_errors_rf.csv'

# Save the DataFrame to the specified path
dm_test_data.to_csv(file_path, index=False)

print(f"Test errors and Newey-West statistics saved to {file_path}")


Running GridSearchCV for Call Options...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END max_depth=15, max_features=15, min_samples_leaf=1, min_samples_split=3, n_estimators=150; total time=  35.2s
[CV] END max_depth=15, max_features=15, min_samples_leaf=1, min_samples_split=3, n_estimators=150; total time=  37.3s
[CV] END max_depth=15, max_features=15, min_samples_leaf=1, min_samples_split=3, n_estimators=150; total time=  37.6s
[CV] END max_depth=15, max_features=15, min_samples_leaf=1, min_samples_split=3, n_estimators=150; total time=  38.0s
[CV] END max_depth=15, max_features=15, min_samples_leaf=1, min_samples_split=3, n_estimators=150; total time=  39.1s


/var/folders/b4/wn87d6495j1_vytb8m33q3mm0000gn/T/ipykernel_18941/2311389172.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  nw_std_error = ols_model.bse[0]  # Newey-West standard error


Call Options Newey-West Standard Error: 0.0010
Running GridSearchCV for Put Options...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  37.5s
[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  39.6s
[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  40.1s
[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  41.2s
[CV] END max_depth=15, max_features=15, min_samples_leaf=4, min_samples_split=2, n_estimators=150; total time=  42.2s
Put Options Newey-West Standard Error: 0.0010
Test errors and Newey-West statistics saved to /Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Notebooks/performence_evaluation/dm_test_errors_rf.csv


/var/folders/b4/wn87d6495j1_vytb8m33q3mm0000gn/T/ipykernel_18941/2311389172.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  nw_std_error = ols_model.bse[0]  # Newey-West standard error


In [ ]:
# import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.model_selection import GridSearchCV

# # Assuming 'data_train' and 'data_test' are already defined and loaded

# # Prepare train data for Call and Put options
# data_train_c = data_train[data_train['cp_flag'] == 'C']
# data_train_p = data_train[data_train['cp_flag'] == 'P']

# # Prepare test data for Call and Put options
# data_test_c = data_test[data_test['cp_flag'] == 'C']
# data_test_p = data_test[data_test['cp_flag'] == 'P']

# def prepare_data_with_gridsearch(train_data, test_data, option_type):
#     """
#     Prepare the data, train Random Forest using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
#     Additionally, select the best models with in-sample R² closest to 0.80 and 0.85.
    
#     Parameters:
#     train_data (pd.DataFrame): The training dataset.
#     test_data (pd.DataFrame): The testing dataset.
#     option_type (str): Call or Put option type for labeling the print output.
#     """
#     # Prepare the train and test data
#     X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
#     y_train = train_data['impl_volatility']  # Training target
    
#     X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
#     y_test = test_data['impl_volatility']  # Test target
    
#     # Define the parameter grid for GridSearchCV
#     param_grid = {
#         'max_features': [15, 20, 25],                 # Use fewer features to reduce model complexity
#         'n_estimators': [100, 200, 300],             # Increase number of trees for better generalization
#         'min_samples_leaf': [5, 10, 15],             # Ensure minimum leaf size is larger to reduce overfitting
#         'min_samples_split': [2, 5],                 # Ensure reasonable splits, prevent unnecessary splits
#         'max_depth': [15, None]                 # Limit the depth of trees to prevent overfitting
#     }    

#     # Initialize the RandomForestRegressor
#     rf_model = RandomForestRegressor()

#     # Initialize GridSearchCV
#     grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
#                                cv=2, n_jobs=-1, verbose=2)

#     # Train the model using GridSearchCV
#     print(f"Running GridSearchCV for {option_type} Options...")
#     grid_search.fit(X_train, y_train)

#     # Initialize tracking variables for the models closest to r² = 0.80 and r² = 0.85
#     closest_to_80 = {'model': None, 'params': None, 'r2_train': float('inf')}
#     closest_to_85 = {'model': None, 'params': None, 'r2_train': float('inf')}

#     # Iterate over the results and select the best models with in-sample R² closest to 0.80 and 0.85
#     for idx, params in enumerate(grid_search.cv_results_['params']):
#         # Get the estimator with the current parameters
#         model = grid_search.estimator.set_params(**params)
#         model.fit(X_train, y_train)

#         # In-sample (training set) predictions
#         y_train_pred = model.predict(X_train)
#         r2_train = r2_score(y_train, y_train_pred)

#         # Track the model closest to r² = 0.80
#         if abs(r2_train - 0.80) < abs(closest_to_80['r2_train'] - 0.80):
#             closest_to_80['model'] = model
#             closest_to_80['params'] = params
#             closest_to_80['r2_train'] = r2_train
        
#         # Track the model closest to r² = 0.85
#         if abs(r2_train - 0.85) < abs(closest_to_85['r2_train'] - 0.85):
#             closest_to_85['model'] = model
#             closest_to_85['params'] = params
#             closest_to_85['r2_train'] = r2_train

#     # Evaluate models closest to r² = 0.80 and r² = 0.85
#     for target_r2, model_info in zip([0.80, 0.85], [closest_to_80, closest_to_85]):
#         if model_info['model'] is None:
#             print(f"\nNo suitable model found for {option_type} Options with in-sample R² closest to {target_r2}.")
#             continue
        
#         # Print the best parameters
#         print(f"\nBest Parameters for {option_type} Options (R² closest to {target_r2}): {model_info['params']}")
#         print(f"In-Sample R² (Training): {model_info['r2_train']:.4f}")

#         # Out-of-sample (test set) predictions
#         y_test_pred = model_info['model'].predict(X_test)

#         # Evaluate Out-of-Sample Performance (on Test Data)
#         rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
#         r2_test = r2_score(y_test, y_test_pred)

#         # Print the results
#         print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
#         print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# # Call the function for Call options data
# prepare_data_with_gridsearch(data_train_c, data_test_c, 'Call')

# # Call the function for Put options data
# prepare_data_with_gridsearch(data_train_p, data_test_p, 'Put')

# print("hello")
